# Project 2 Team 3 Technical Document 

The aim  this project was to develop a data resource that enables analysis of COVID cases by postcode and Local Government Area (LGA) compared to the population and population density of each LGA.

Cleaned df with help of Pandas Drop and Filter Functions, we also used the Python Lambda function to split infomation from one column and create two new columns. We also used a DataFrame.rename to get correct column names.  - line number XX change / add 

Data Sources:

Three data sets were sourced to undertake this analysis

1. Commonwealth Australian Beureau of Statisitcs (ABS), population by LGA
2. LGA Mapping file 
3. The Victorian DHHS COVID data:

1. Commonwealth Australian Beureau of Statisitcs (ABS), population by LGA
        (a) data soure - url:"https://api.data.abs.gov.au/data/C21_G04_LGA/....2"
        (b) Format: Data set sourced via API
        (c) Extract and Load method: Sourced via abs API 
        (d) Original data fields: DATAFLOW,	AGEINGP: Age,	SEXP: Sex, 	REGION: Region,	REGION_TYPE: Region Type	        STATE: State,	TIME_PERIOD: Time Period,	OBS_VALUE
        (e) Data transformation: 
              (i) REGION: Region:  separate data in REGION: Region column into 'Regions ID' and 'Region Name'
              (ii) Drop DATAFLOW, AGEINGP: Age, REGION: Region, STATE: State
              (iii) SEXP: Sex filter and retain 3: persons value in SEXP: Sex column - this removed the male and                       female categores and enabled to calculation of the total population
              (v)Rename 'OBS_VALUE' to 'Population'
              (vi) Drop 'persons' column to avoid duplicating data         
        (f) Comments
                This data set could not be directly merged with the Victorian DHHS COVID data set. The                             'Region Name' field in this data set did not include suffic (x) that was present in the                           'Localgovernmentarea' field. 
        (g) Final data fields: Population	Region ID (PRIMARY KEY)	Region Name
 
        
 2. LGA Mapping and LGA SQKM file 
         (a) data source: Resource/'LGA_2020_VIC2.csv'
         (b) Format: CSV 
         (c) Extract and load method: data downloaded from Vic Government websit 
         (d) Original Data Fields: MB_CODE_2016	LGA_CODE_2020	LGA_NAME_2020	STATE_CODE_2016	STATE_NAME_2016	                AREA_ALBERS_SQKM
         (e) Data transformation
             (i) Drop MB_CODE_2016 STATE_CODE_2016	STATE_NAME_2016	
         (f) Comments 
             This data set was required to map the LGAs in the abs and COVID data sets. This data set also provides the area square kilometer data to enable population density for each LGA to be shown
         (g) Final data fields: LGA_CODE_2020 int (PRIMARY KEY)	LGA_NAME_20201 object (FORIEGN KEY)	                            AREA_ALBERS_SQKM float

1. The Victorian DHHS COVID that shows the number of COVID cases by LGA by date (day, month and year) 
        (a) data source: "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-lga-source-csv"
        (b) Format: CSV 
        (c) Extract and Load method: read in directly from the DHHS web site as too large to download and save
        (d) Original data fields: diagnosis_date,Postcode,	acquired,	Localgovernmentarea.
        (e) Data transformation: 
              (i) use the 'diagnosis_date' to create a column called 'year' to enable the volume of people to be                       grouped by year 
              (ii) drop the 'acquired' field
        (f) Comments: 
                This data set could not be directly merged with the abs population data set. The                                   'Localgovernmentarea' field in this data set did not include the LGA numerical ID and contained an                  additional code in the form of(x)at the end of the name. This code was not present in the abs                      data set to enable data to be merged a third data set is required to match the LGA name and ID .                  This data set did not show population density 
         (g) Final Data Fields 

This resource forms the foundations which, at a future date, can enable the addition and analysis of other socio, demographic economic factors that are captured in Census data and other State and Commonwealth data sets

SQL

ERD Diagram 


In [562]:
#import dependencies 
import json
import requests
import pandas as pd
from io import StringIO
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [563]:
# Review abs API documentation and liase with abs data custodians to identify required data points to source
# population data data from abs API 
url = "https://api.data.abs.gov.au/data/C21_G04_LGA/....2"

In [564]:
#Source Population data from ABS API and review data field and data in top 5 rows
headers = {"Accept": "application/vnd.sdmx.data+csv;labels=both"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)

abs_df = pd.read_csv(data)
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970


In [565]:
#split REGION: Region field to create 'Region ID' field 
abs_df["Region ID"]=abs_df["REGION: Region"].map(lambda x:x.split(":")[0])
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE,Region ID
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459,24210
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764,24410
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759,25250
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89,25430
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970,25710


In [566]:
#split REGION: Region field to create 'Region Name' field 
abs_df["Region Name"]=abs_df["REGION: Region"].map(lambda x:x.split(":")[1])
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE,Region ID,Region Name
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459,24210,Manningham
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764,24410,Maroondah
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759,25250,Moreland
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89,25430,Mount Alexander
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970,25710,Nillumbik


In [567]:
#drop round of unwanted items from abs_df

abs_df.drop(columns=["DATAFLOW","AGEINGP: Age","REGION_TYPE: Region Type","STATE: State","TIME_PERIOD: Time Period"], inplace= True)
abs_df.head()



,SEXP: Sex,REGION: Region,OBS_VALUE,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
1,2: Females,24410: Maroondah,764,24410,Maroondah
2,1: Males,25250: Moreland,759,25250,Moreland
3,1: Males,25430: Mount Alexander,89,25430,Mount Alexander
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik


In [568]:
#Sex filter and retain 3: persons value in SEXP: Sex column - 
#this removed the male and female categores and enabled to calculation of the total population
abs_clean_df=abs_df[abs_df["SEXP: Sex"] == '3: Persons'] 
abs_clean_df.head()

,SEXP: Sex,REGION: Region,OBS_VALUE,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik
5,3: Persons,25900: Port Phillip,2467,25900,Port Phillip
6,3: Persons,26260: Southern Grampians,852,26260,Southern Grampians
9,3: Persons,29399: Unincorporated Vic,7,29399,Unincorporated Vic


In [569]:
# abs_clean_df.describe()

In [570]:
# abs_clean_df.info()

In [571]:
#Rename 'OBS_VALUE' to 'Population' dfmi.loc[:, ('one', 'second')]

abs_clean_df.rename(columns={'OBS_VALUE': 'Population'}, inplace=True)
abs_clean_df.head()


,SEXP: Sex,REGION: Region,Population,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik
5,3: Persons,25900: Port Phillip,2467,25900,Port Phillip
6,3: Persons,26260: Southern Grampians,852,26260,Southern Grampians
9,3: Persons,29399: Unincorporated Vic,7,29399,Unincorporated Vic


In [572]:
#Drop 'persons' column to avoid duplicating data
abs_clean_df.drop(columns=["SEXP: Sex","REGION: Region"], inplace= True)
abs_clean_df.head()



,Population,Region ID,Region Name
0,1459,24210,Manningham
4,970,25710,Nillumbik
5,2467,25900,Port Phillip
6,852,26260,Southern Grampians
9,7,29399,Unincorporated Vic


In [573]:
abs_clean_df= abs_clean_df.drop_duplicates(subset=['Region ID'],keep='first')


In [574]:
abs_clean_df.head()

,Population,Region ID,Region Name
0,1459,24210,Manningham
4,970,25710,Nillumbik
5,2467,25900,Port Phillip
6,852,26260,Southern Grampians
9,7,29399,Unincorporated Vic


In [575]:
# abs_clean_df.describe()

In [576]:
# abs_clean_df.info()

In [577]:
#Convert abs_clean data set to a csv format
abs_clean_df.to_csv("abs_clean.csv")

In [578]:
# need to check unnamed field 
abs_clean=pd.read_csv("abs_clean.csv")
abs_clean.head()

,Unnamed: 0,Population,Region ID,Region Name
0,0,1459,24210,Manningham
1,4,970,25710,Nillumbik
2,5,2467,25900,Port Phillip
3,6,852,26260,Southern Grampians
4,9,7,29399,Unincorporated Vic


In [579]:
#???
# abs_clean_df = abs_clean_df.apply(lambda x: "{:},".format(x))


In [580]:
abs_clean_df.head()

,Population,Region ID,Region Name
0,1459,24210,Manningham
4,970,25710,Nillumbik
5,2467,25900,Port Phillip
6,852,26260,Southern Grampians
9,7,29399,Unincorporated Vic


In [581]:
# Read in  LGA Mapping and LGA SQKM file
lga_mapping_df = pd.read_csv('../Data/LGA_2020_VIC2.csv')
lga_mapping_df.head()

,MB_CODE_2016,LGA_CODE_2020,LGA_NAME_2020,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
0,20000009499,29499,No usual address (Vic.),2,Victoria,0.0000
1,20000010000,20110,Alpine (S),2,Victoria,0.0127
2,20000021000,20110,Alpine (S),2,Victoria,0.0127
3,20000022000,20110,Alpine (S),2,Victoria,0.0105
4,20000023000,20110,Alpine (S),2,Victoria,0.0105


In [582]:
#drop unwnated items from df1
lga_mapping_df.drop(columns=["MB_CODE_2016","STATE_CODE_2016","STATE_NAME_2016"], inplace= True)
lga_mapping_df.head()



,LGA_CODE_2020,LGA_NAME_2020,AREA_ALBERS_SQKM
0,29499,No usual address (Vic.),0.0000
1,20110,Alpine (S),0.0127
2,20110,Alpine (S),0.0127
3,20110,Alpine (S),0.0105
4,20110,Alpine (S),0.0105


In [583]:
lga_mapping_df.describe()

,LGA_CODE_2020,AREA_ALBERS_SQKM
count,85014.000000,85014.000000
mean,23904.760357,2.675978
std,2117.225178,27.492022
min,20110.000000,0.000000
25%,22170.000000,0.021900
50%,23670.000000,0.034800
75%,25430.000000,0.068400
max,29799.000000,5172.342600


In [584]:
lga_mapping_df= lga_mapping_df.drop_duplicates(subset=['LGA_CODE_2020'],keep='first')

In [585]:
# lga_mapping_df.info()

In [586]:
#Convert dataframe to csv

lga_mapping_df.to_csv("LGA_CLEAN.csv")

In [587]:
#Importing The Victorian DHHS COVID data

dhhs_covid_df = pd.read_csv("https://www.dhhs.vic.gov.au/ncov-covid-cases-by-lga-source-csv")
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea
0,2020-03-11,3217,Travel overseas,Greater Geelong (C)
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S)
2,2020-03-09,3206,Travel overseas,Port Phillip (C)
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C)
4,2020-03-01,3802,Travel overseas,Casey (C)


In [588]:
#add in  'year' column
dhhs_covid_df["year"]=dhhs_covid_df["diagnosis_date"].map(lambda x:x[0:4])

In [589]:
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea,year
0,2020-03-11,3217,Travel overseas,Greater Geelong (C),2020
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S),2020
2,2020-03-09,3206,Travel overseas,Port Phillip (C),2020
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C),2020
4,2020-03-01,3802,Travel overseas,Casey (C),2020


In [590]:
dhhs_covid_df["month"]=dhhs_covid_df["diagnosis_date"].map(lambda x:x[5:7])

In [591]:
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea,year,month
0,2020-03-11,3217,Travel overseas,Greater Geelong (C),2020,03
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S),2020,03
2,2020-03-09,3206,Travel overseas,Port Phillip (C),2020,03
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C),2020,08
4,2020-03-01,3802,Travel overseas,Casey (C),2020,03


In [592]:
# dhhs_covid_df.info()

In [593]:
# dhhs_covid_df.describe()


In [594]:
#Convert dataframe to csv

dhhs_covid_df.to_csv("COVID_CLEAN.csv")

In [595]:
from sqlalchemy import create_engine

In [596]:
csv_file = "abs_clean.csv"
abs_df = pd.read_csv(csv_file)
abs_df.head()


,Unnamed: 0,Population,Region ID,Region Name
0,0,1459,24210,Manningham
1,4,970,25710,Nillumbik
2,5,2467,25900,Port Phillip
3,6,852,26260,Southern Grampians
4,9,7,29399,Unincorporated Vic


In [597]:
csv_file = "COVID_CLEAN.csv"
covid_df = pd.read_csv(csv_file)
covid_df.head()

,Unnamed: 0,diagnosis_date,Postcode,acquired,Localgovernmentarea,year,month
0,0,2020-03-11,3217,Travel overseas,Greater Geelong (C),2020,3
1,1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S),2020,3
2,2,2020-03-09,3206,Travel overseas,Port Phillip (C),2020,3
3,3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C),2020,8
4,4,2020-03-01,3802,Travel overseas,Casey (C),2020,3


In [598]:
csv_file = "LGA_CLEAN.csv"
lga_df = pd.read_csv(csv_file)
lga_df.head()

,Unnamed: 0,LGA_CODE_2020,LGA_NAME_2020,AREA_ALBERS_SQKM
0,0,29499,No usual address (Vic.),0.0000
1,1,20110,Alpine (S),0.0127
2,257,20260,Ararat (RC),0.0394
3,457,25490,Moyne (S),19.2135
4,488,20570,Ballarat (C),0.0437


In [599]:
#connect to local database
rds_connection_string = "postgres:Fedelma22!@localhost:5432/Project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [600]:
engine.table_names()

['abs_clean', 'lga_clean', 'covid_clean']

In [602]:
abs_df.to_sql(name='ABS_CLEAN', con=engine, if_exists='append', index=False)
lga_df.to_sql(name='LGA_CLEAN', con=engine, if_exists='append', index=False)
covid_df.to_sql(name='COVID_CLEAN',con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from  ', con=engine).head()